In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [6]:
data = pd.read_csv('data_final.csv')
# Keeping only the neccessary columns
data = data[['Headline','Price movement']]

In [8]:
data = data[data['Price movement'] != "#N/A"]
data['Headline'] = data['Headline'].apply(lambda x: x.lower())
data['Headline'] = data['Headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[data['Price movement'] == 'Up'].size)
print(data[data['Price movement'] == 'Down'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Headline'].values)
X = tokenizer.texts_to_sequences(data['Headline'].values)
X = pad_sequences(X)

5308
4338


In [28]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 34, 128)           256000    
                                                                 
 conv1d_9 (Conv1D)           (None, 34, 128)           65664     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 17, 128)          0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 17, 64)            32832     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 8, 64)            0         
 1D)                                                             
                                                                 
 conv1d_11 (Conv1D)          (None, 8, 32)            

In [39]:
Y = pd.get_dummies(data['Price movement']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3327, 34) (3327, 2)
(1640, 34) (1640, 2)


In [45]:
batch_size = 128
model.fit(X_train, Y_train, epochs=7, batch_size=batch_size, verbose = 2)

Epoch 1/7
26/26 - 0s - loss: 993256832.0000 - accuracy: 0.4896 - 490ms/epoch - 19ms/step
Epoch 2/7
26/26 - 0s - loss: 652632832.0000 - accuracy: 0.4878 - 493ms/epoch - 19ms/step
Epoch 3/7
26/26 - 0s - loss: 912826624.0000 - accuracy: 0.5047 - 481ms/epoch - 18ms/step
Epoch 4/7
26/26 - 0s - loss: 1084959360.0000 - accuracy: 0.4944 - 479ms/epoch - 18ms/step
Epoch 5/7
26/26 - 0s - loss: 1403197440.0000 - accuracy: 0.5011 - 493ms/epoch - 19ms/step
Epoch 6/7
26/26 - 0s - loss: 2068391424.0000 - accuracy: 0.5080 - 480ms/epoch - 18ms/step
Epoch 7/7
26/26 - 0s - loss: 3190924288.0000 - accuracy: 0.5095 - 498ms/epoch - 19ms/step


In [46]:
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

52/52 - 0s - loss: 2597113600.0000 - accuracy: 0.5360 - 144ms/epoch - 3ms/step
Accuracy: 53.60%
